In [64]:
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt


## importer les données


In [65]:
#importer mes donnees 
data = pd.read_csv("SuperMarket Analysis.csv")

#afficher les 5 premieres lignes 
data.head()


,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Sales,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,Alex,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,1:08:00 PM,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,Giza,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29:00 AM,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,Alex,Yangon,Normal,Female,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,1:23:00 PM,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,Alex,Yangon,Member,Female,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,8:33:00 PM,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,Alex,Yangon,Member,Female,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37:00 AM,Ewallet,604.17,4.761905,30.2085,5.3


In [66]:
#lignes et colonnes manquantes 
lignes_manquantes = data.isnull().sum(axis=1)
colonnes_manquantes = data.isnull().sum()
print(f"les lignes manquantes sont : {lignes_manquantes}")
print(f"les colonnes manquantes sont : {colonnes_manquantes}")

#voir les doublons
doublons = data.duplicated().sum() 
print(f"les doublons sont : {doublons}") 

#supprimer les doublons
data = data.drop_duplicates() 



#verifier que les colonnes int contiennent des valeurs numeriques 
data["Unit price"] = pd.to_numeric(data["Unit price"], errors='coerce') 
data["Quantity"] = pd.to_numeric(data["Quantity"], errors='coerce') 
data["Sales"] = pd.to_numeric(data["Sales"], errors='coerce')
data["Rating"] = pd.to_numeric(data["Rating"], errors='coerce')



les lignes manquantes sont : 0      0
1      0
2      0
3      0
4      0
      ..
995    0
996    0
997    0
998    0
999    0
Length: 1000, dtype: int64
les colonnes manquantes sont : Invoice ID                 0
Branch                     0
City                       0
Customer type              0
Gender                     0
Product line               0
Unit price                 0
Quantity                   0
Tax 5%                     0
Sales                      0
Date                       0
Time                       0
Payment                    0
cogs                       0
gross margin percentage    0
gross income               0
Rating                     0
dtype: int64
les doublons sont : 0


In [70]:
#avoir les donnes aberantes avec la methode interquantile 
#fonction detecter les valeurs aberantes 
def valeur_manquante(df , colonne) :
    q1 = df[colonne].quantile(0.25)
    q3 = df[colonne].quantile(0.75)
    iqr = q3 - q1
    born_sup = q3 + 1.5 * iqr
    born_inf = q1 - 1.5 * iqr
    return df[(df[colonne] < born_inf) | (df[colonne] > born_sup)][colonne] 

#suuprimer les valeurs aberantes pour les colonnes numeriques
for col in data.select_dtypes(include=['int64', 'float64']).columns:
    data = data[~((data[col] < data[col].quantile(0.25)) | (data[col] > data[col].quantile(0.75)))] #supprimer les valeurs aberantes
    
 
#changer la colonne en Date 
data["Date"] = pd.to_datetime(data["Date"] , errors='coerce')
print(data["Date"].dtypes)


#verifier que sales est egal a la multiplication de unit price et quantity + tax 5% pour toute les lignes
if data['Sales'].equals((data['Unit price'] * data['Quantity']) + (data['Tax 5%'] * data['Quantity'])):
    print("Sales est egal a la multiplication de unit price et quantity + tax 5% pour toute les lignes")
    
    


     
    

datetime64[ns]
Sales est egal a la multiplication de unit price et quantity + tax 5% pour toute les lignes


,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Sales,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
183,824-88-3614,Giza,Naypyitaw,Normal,Male,Health and beauty,34.31,8,13.724,288.204,2019-01-25,3:00:00 PM,Ewallet,274.48,4.761905,13.724,5.7
189,593-65-1552,Giza,Naypyitaw,Normal,Female,Home and lifestyle,69.81,4,13.962,293.202,2019-01-28,8:50:00 PM,Credit card,279.24,4.761905,13.962,5.9
386,689-16-9784,Giza,Naypyitaw,Normal,Male,Food and beverages,46.77,6,14.031,294.651,2019-03-11,1:37:00 PM,Cash,280.62,4.761905,14.031,6.0
538,308-81-0538,Alex,Yangon,Normal,Female,Fashion accessories,73.05,4,14.610,306.810,2019-02-25,5:16:00 PM,Credit card,292.20,4.761905,14.610,4.9
677,274-05-5470,Alex,Yangon,Member,Female,Food and beverages,73.47,4,14.694,308.574,2019-02-23,6:30:00 PM,Cash,293.88,4.761905,14.694,6.0
